### Adjust Word Group Repeat With Youtube Link

In [34]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [35]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [36]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 5000  # 28 native word end index

# file extention
file_ext = "Threegram"

In [37]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [38]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 3/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [39]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Result/2-Find Word Group Youtube Link"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [40]:
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel

,search_string,start_time,end_time,video_id
0,ama yine de,248,250,Y_stiGI98IA
1,ama yine de,27,29,OjhjTSrBeCs
2,ama yine de,61,63,r_9ngjGh2Ks
3,ama yine de,472,474,MeHdQsEjfjQ
4,ama yine de,1559,1562,qM0a2o6BpZM
...,...,...,...,...
102,öyle bir şey,2941,2943,4C_h8ZyIXdc
103,öyle bir şey,3834,3836,BfX0BJRrRhI
104,öyle bir şey,3033,3035,ccnwNHUwrP0
105,öyle bir şey,3658,3660,OTDkvCGQyVQ


In [41]:
df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
df_link_selected_manuel

,search_string,start_time,end_time,video_id,video_url
0,ama yine de,248,250,Y_stiGI98IA,https://www.youtube.com/watch?v=Y_stiGI98IA&t=...
1,ama yine de,27,29,OjhjTSrBeCs,https://www.youtube.com/watch?v=OjhjTSrBeCs&t=27s
2,ama yine de,61,63,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=61s
3,ama yine de,472,474,MeHdQsEjfjQ,https://www.youtube.com/watch?v=MeHdQsEjfjQ&t=...
4,ama yine de,1559,1562,qM0a2o6BpZM,https://www.youtube.com/watch?v=qM0a2o6BpZM&t=...
...,...,...,...,...,...
102,öyle bir şey,2941,2943,4C_h8ZyIXdc,https://www.youtube.com/watch?v=4C_h8ZyIXdc&t=...
103,öyle bir şey,3834,3836,BfX0BJRrRhI,https://www.youtube.com/watch?v=BfX0BJRrRhI&t=...
104,öyle bir şey,3033,3035,ccnwNHUwrP0,https://www.youtube.com/watch?v=ccnwNHUwrP0&t=...
105,öyle bir şey,3658,3660,OTDkvCGQyVQ,https://www.youtube.com/watch?v=OTDkvCGQyVQ&t=...


In [42]:
df_count = df_link_selected_manuel.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,böyle bir şey,8
1,bir kez daha,8
2,dalga mı geçiyorsun,7
3,ne yazık ki,7
4,önemli bir şey,6
5,hiçbir şey yok,6
6,öyle bir şey,6
7,bir fikrim var,6
8,bir şey daha,5
9,bir şey mi,5


In [43]:
df_count = df_count[df_count["count"] > 2]

In [44]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,video_id,video_url,count
0,böyle bir şey,1109,1111,QLQAjiKlG5Q,https://www.youtube.com/watch?v=QLQAjiKlG5Q&t=...,8
1,böyle bir şey,578,580,IAiCYOTv9XE,https://www.youtube.com/watch?v=IAiCYOTv9XE&t=...,8
2,böyle bir şey,577,579,HQoyco1OE18,https://www.youtube.com/watch?v=HQoyco1OE18&t=...,8
3,böyle bir şey,723,725,XnLdFB3Zfw0,https://www.youtube.com/watch?v=XnLdFB3Zfw0&t=...,8
4,böyle bir şey,3467,3469,7G29Odeyqpk,https://www.youtube.com/watch?v=7G29Odeyqpk&t=...,8
...,...,...,...,...,...,...
101,bu kadar mı,2624,2626,UtcBPDTBiko,https://www.youtube.com/watch?v=UtcBPDTBiko&t=...,3
102,bu kadar mı,3872,3874,gVOllvdKurs,https://www.youtube.com/watch?v=gVOllvdKurs&t=...,3
103,başka bir şey,462,464,zpPamhZvbq8,https://www.youtube.com/watch?v=zpPamhZvbq8&t=...,3
104,başka bir şey,530,532,eqEmUZF5LQY,https://www.youtube.com/watch?v=eqEmUZF5LQY&t=...,3


In [45]:
df_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Data/Deployment/Talk Time 3 {file_ext} Video List.xlsx", index=False)

In [46]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx", index=False)

##### Link Repeat

In [47]:
path_folder_file2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Result/2-Find Word Group Youtube Link/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx"
sheet2 = "Sheet1"  # Sheet1

In [49]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,video_id,video_url,count
0,böyle bir şey,1109,1111,QLQAjiKlG5Q,https://www.youtube.com/watch?v=QLQAjiKlG5Q&t=...,7
1,böyle bir şey,578,580,IAiCYOTv9XE,https://www.youtube.com/watch?v=IAiCYOTv9XE&t=...,7
2,böyle bir şey,577,579,HQoyco1OE18,https://www.youtube.com/watch?v=HQoyco1OE18&t=...,7
3,böyle bir şey,3467,3469,7G29Odeyqpk,https://www.youtube.com/watch?v=7G29Odeyqpk&t=...,7
4,böyle bir şey,943,945,iMj5gB0XHRw,https://www.youtube.com/watch?v=iMj5gB0XHRw&t=...,7
...,...,...,...,...,...,...
99,bu kadar mı,2624,2626,UtcBPDTBiko,https://www.youtube.com/watch?v=UtcBPDTBiko&t=...,3
100,bu kadar mı,3872,3874,gVOllvdKurs,https://www.youtube.com/watch?v=gVOllvdKurs&t=...,3
101,başka bir şey,462,464,zpPamhZvbq8,https://www.youtube.com/watch?v=zpPamhZvbq8&t=...,3
102,başka bir şey,530,532,eqEmUZF5LQY,https://www.youtube.com/watch?v=eqEmUZF5LQY&t=...,3


In [50]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

20

In [51]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [52]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [53]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [54]:
df_all_join_result

,video_id,start_time,end_time,search_string,count,web_format
0,"QLQAjiKlG5Q,IAiCYOTv9XE,HQoyco1OE18,7G29Odeyqp...","1109,578,577,3467,943,2711,306,2711,5,2711,5,2...","1111,580,579,3469,945,2713,308,2713,8,2713,8,2...",böyle bir şey,7.0,"[play_lms_videos video_id=QLQAjiKlG5Q,IAiCYOTv..."
1,"1KN8foDEwQs,NRmpxmWT6Q8,LwmbPWjegxQ,F4pAIWvIlw...","656,522,753,488,370,273,112,753,5,753,5,753,5","658,524,755,490,372,275,114,755,8,755,8,755,8",bir kez daha,7.0,"[play_lms_videos video_id=1KN8foDEwQs,NRmpxmWT..."
2,"ohTFAhx3kKo,hyEcuOeUWjY,uCy3SFC9S3E,b7iTm_RLtE...","903,66,864,514,217,459,177,217,5,217,5,217,5","906,68,866,516,219,461,179,219,8,219,8,219,8",ne yazık ki,7.0,"[play_lms_videos video_id=ohTFAhx3kKo,hyEcuOeU..."
3,"FO07ssbT9bA,AjQkYpuB6Dc,UDlsIakb9Kg,SzFGh9iLnd...","1565,8342,7068,417,5677,84,3454,8342,5,8342,5,...","1567,8344,7070,419,5679,86,3456,8344,8,8344,8,...",dalga mı geçiyorsun,7.0,"[play_lms_videos video_id=FO07ssbT9bA,AjQkYpuB..."
4,"gQg3cyWahgc,4C_h8ZyIXdc,BfX0BJRrRhI,ccnwNHUwrP...","3332,2941,3834,3033,3658,6797,3658,5,3658,5,36...","3334,2943,3836,3035,3660,6799,3660,8,3660,8,36...",öyle bir şey,6.0,"[play_lms_videos video_id=gQg3cyWahgc,4C_h8ZyI..."
5,"PLMQWYGJuVQ,3ogB_jaW4FI,JvsqNlgYDFc,24MOyiODOD...","78,3010,8120,3600,1828,1168,1168,5,1168,5,1168,5","80,3012,8122,3602,1830,1170,1170,8,1170,8,1170,8",önemli bir şey,6.0,"[play_lms_videos video_id=PLMQWYGJuVQ,3ogB_jaW..."
6,"XLFR3Sew2b0,qdS3A__QRBA,QtOjRnw10Ig,oKfaa43c0h...","1058,199,297,14,275,349,297,5,297,5,297,5","1062,201,299,16,277,351,299,8,299,8,299,8",hiçbir şey yok,6.0,"[play_lms_videos video_id=XLFR3Sew2b0,qdS3A__Q..."
7,"gxynhdL6m4k,gE85fd99w44,J3ZzEixVXKs,Oz7FkuxsmJ...","3953,6406,7705,4916,859,89,4916,5,4916,5,4916,5","3956,6408,7707,4918,861,92,4918,8,4918,8,4918,8",bir fikrim var,6.0,"[play_lms_videos video_id=gxynhdL6m4k,gE85fd99..."
8,"x6aEnqpTUBo,rovTz6ejwvc,tBRYOWcZkZ0,Ez9tXJrcag...","3716,541,194,6621,458,194,5,194,5,194,5","3718,543,196,6623,460,196,8,196,8,196,8",ne olursa olsun,5.0,"[play_lms_videos video_id=x6aEnqpTUBo,rovTz6ej..."
9,"37Ru98CTHkE,ncy57YkTtdU,rUyOsgYjbFI,86of7l2p8K...","304,3320,427,158,458,427,5,427,5,427,5","306,3322,429,160,460,429,8,429,8,429,8",iyi bir fikir,5.0,"[play_lms_videos video_id=37Ru98CTHkE,ncy57YkT..."


#### Copy Move And Delete

In [55]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_5000_Word_Youtube_Talk_Time_Join_Threegram.xlsx']

In [56]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [57]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

In [ ]:
#df_test["web_format"] = f"[play_lms_videos video_id="+df_test["video_id"].map(str)+" start_time="+df_test["start_time"].map(str)+" end_time="+df_test["end_time"].map(str)+"]"
#df_test